In [7]:
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np

# Pre-trained cross encoder
model = CrossEncoder('cross-encoder/stsb-distilroberta-base')

# We want to compute the similarity between the query sentence
query = 'A man is eating pasta.'

# With all sentences in the corpus
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'A cheetah is running behind its prey.'
          ]

# So we create the respective sentence combinations
sentence_combinations = [[query, corpus_sentence] for corpus_sentence in corpus]

# Compute the similarity scores for these combinations
similarity_scores = model.predict(sentence_combinations)

# Sort the scores in decreasing order
sim_scores_argsort = reversed(np.argsort(similarity_scores))

# Print the scores
print("Query:", query)
for idx in sim_scores_argsort:
    print("{:.2f}\t{}".format(similarity_scores[idx], corpus[idx]))

Query: A man is eating pasta.
0.67	A man is eating food.
0.34	A man is eating a piece of bread.
0.08	A man is riding a horse.
0.07	A man is riding a white horse on an enclosed ground.
0.01	The girl is carrying a baby.
0.01	Two men pushed carts through the woods.
0.01	A monkey is playing drums.
0.01	A woman is playing violin.
0.01	A cheetah is running behind its prey.


Ok ça fonctionne bien

Les modèles du papier sont dispo [ici](https://sbert.net/docs/pretrained-models/ce-msmarco.html) et [ici](https://www.sbert.net/docs/pretrained_cross-encoders.html). On peut mettre n'importe quel model de Hugging face je pense
* nboost/pt-bert-base-uncased-msmarco
* nboost/pt-bert-large-msmarco
* cross-encoder/stsb-distilroberta-base
* Plusieurs mini LM de Dispo. Avec un nombre de layer différent, j'ai pas trouvé dans le papier lequel est utilisé

In [8]:
from src.bm25 import BM25, min_max_local
bm25 = BM25().fit(corpus)
queries = ['A man is eating pasta.']
queries_score = bm25.predict(queries)
queries_score_normalized = (min_max_local(queries_score)*100).astype(int).astype(str)
queries_score_normalized

array([['100', '81', '16', '42', '35', '0', '31', '35', '30']],
      dtype='<U21')

In [17]:
sentence_combinations = [
    # ["[CLS]", query, "[SEP]", bm25_score, "[SEP]", corpus_sentence]
    [query, bm25_score, corpus_sentence]
    for corpus_sentence, bm25_score in zip(corpus, queries_score_normalized[0])
]
sentence_combinations

[['A man is eating pasta.', '100', 'A man is eating food.'],
 ['A man is eating pasta.', '81', 'A man is eating a piece of bread.'],
 ['A man is eating pasta.', '16', 'The girl is carrying a baby.'],
 ['A man is eating pasta.', '42', 'A man is riding a horse.'],
 ['A man is eating pasta.', '35', 'A woman is playing violin.'],
 ['A man is eating pasta.', '0', 'Two men pushed carts through the woods.'],
 ['A man is eating pasta.',
  '31',
  'A man is riding a white horse on an enclosed ground.'],
 ['A man is eating pasta.', '35', 'A monkey is playing drums.'],
 ['A man is eating pasta.', '30', 'A cheetah is running behind its prey.']]

In [18]:
# Compute the similarity scores for these combinations
similarity_scores = model.predict(sentence_combinations)

# Sort the scores in decreasing order
sim_scores_argsort = reversed(np.argsort(similarity_scores))

# Print the scores
print("Query:", query)
for idx in sim_scores_argsort:
    print("{:.2f}\t{}".format(similarity_scores[idx], corpus[idx]))

/home/charles/.pyenv/versions/3.10.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([9, 14])) that is different to the input size (torch.Size([9])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (9) must match the size of tensor b (14) at non-singleton dimension 1

In [19]:
model.predict()

RobertaConfig {
  "_name_or_path": "cross-encoder/stsb-distilroberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}